In [48]:
import requests
import pandas as pd
import json,os

#sharepoint dev site
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# Token endpoint
url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Get token for Microsoft Graph
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'https://graph.microsoft.com/.default'
}

token_response = requests.post(url, data=data)
access_token = token_response.json()['access_token']

veeva_headers = {
    'Authorization': '4F1A1FD86C5515BF2BD1B7E3D813915FA4FF5292506CBE417C14643810D280B6062AE46FC19B833CEE8E38420659B2B06191A3F4845E5677519896CDBC73F931',  # Add your token
    'Content-Type': 'application/x-www-form-urlencoded',
}
sp_headers = {
        'Authorization': f'Bearer {access_token}',  
        'Content-Type': 'application/octet-stream'
        }
LOCAL_DOWNLOAD_PATH = './'

In [ ]:
def get_lookup():
    query = (
    "SELECT id, name__v, filename__v,specification_main_version__c, specification_minor_version__c, subtype__v, status__v "
    "FROM documents "
    "WHERE (specification_minor_version__c != null OR specification_main_version__c != null) "
    "AND (status__v = 'Approved' OR status__v = 'Obsolete') "
    "AND (subtype__v = 'Formulation Inert Specification' "
    "OR subtype__v = 'Composition of Formulation (CoF)' "
    "OR subtype__v = 'Specification Technical Material' "
    "OR subtype__v = 'Formulation Specification' "
    "OR subtype__v = 'Selection List')"
)

    start = 2000
    limit = 1000
    params = {"q": query,
        "start": start,
        "limit": limit}
        
    base_url = 'https://envu-regulatoryone.veevavault.com/api/v24.2/query'
    response = requests.post(url=base_url, headers=veeva_headers, params=params).json()
    return response
actualsepc = get_lookup()


actualsepc

{'responseStatus': 'SUCCESS',
 'responseDetails': {'pagesize': 1000,
  'pageoffset': 0,
  'size': 1000,
  'total': 4845,
  'next_page': '/api/v24.2/query/246ed475-a52a-4fa3-89ea-7f44f5918464?pagesize=1000&pageoffset=1000'},
 'data': [{'id': 174906,
   'name__v': '102D00000648-01.00',
   'filename__v': 'V-102D00000648-01.00 (174906_0_2).docx',
   'specification_main_version__c': ['V4A00000007A003'],
   'specification_minor_version__c': ['VBM000000003003'],
   'subtype__v': 'Formulation Inert Specification',
   'status__v': 'Approved'},
  {'id': 175102,
   'name__v': '10163643',
   'filename__v': '10163643.PDF',
   'specification_main_version__c': ['V4A00000000Q024'],
   'specification_minor_version__c': None,
   'subtype__v': 'Selection List',
   'status__v': 'Approved'},
  {'id': 175103,
   'name__v': '10318846',
   'filename__v': '10318846.PDF',
   'specification_main_version__c': ['V4A00000000R138'],
   'specification_minor_version__c': None,
   'subtype__v': 'Formulation Inert Speci

In [ ]:
file_download_url = f"https://sb-envu-deployment.veevavault.com/api/v24.2/objects/documents/9801/file"
file_name = 'Label_Albania_Finnish111.pdf'
file_response = requests.get(url=file_download_url,headers=veeva_headers)

print(f"file_response.statuscode:{file_response.status_code}")
drive_id = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
folder_name = 'Shared Documents'
folder_name_encoded = folder_name.replace(" ", "%20") if folder_name else ""


# update_headers={
#     'Authorization': f'Bearer {access_token}',
#     'Content-Type': 'application/json'   # must be JSON
# }


upload_url  = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{file_name}:/content"
upload_response  = requests.put(upload_url , headers=sp_headers, data=file_response.content)

file_response.statuscode:200


In [46]:
upload_response.json()['webUrl']

'https://envu.sharepoint.com/sites/EnvuSpecificationsUAT/Shared%20Documents/Label_Albania_Finnish111.pdf'

File Endpoint

In [40]:
update_url = "https://graph.microsoft.com/v1.0/sites/envu.sharepoint.com:/sites/EnvuSpecificationsProd:/drives"

document_info = []
upload_response  = requests.get(update_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#drives",
   "value": [
      {
         "createdDateTime": "2025-10-17T09:36:03Z",
         "description": "",
         "id": "b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRFiYhc0-ZK6Q6P5lvuiy7R0",
         "lastModifiedDateTime": "2025-10-17T09:39:24Z",
         "name": "Documents",
         "webUrl": "https://envu.sharepoint.com/sites/EnvuSpecificationsProd/Documents",
         "driveType": "documentLibrary",
         "createdBy": {
            "user": {
               "email": "rahul.myadam.ext@envu.com",
               "id": "1d1eeff8-b123-4499-ab3c-049727f3d8a7",
               "displayName": "Rahul Myadam"
            }
         },
         "lastModifiedBy": {
            "user": {
               "email": "rahul.myadam.ext@envu.com",
               "id": "1d1eeff8-b123-4499-ab3c-049727f3d8a7",
               "displayName": "Rahul Myadam"
            }
         },
         "owner": {
            "user": {
           

In [61]:
import requests
import pandas as pd
import json

drive_id = 'b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRFiYhc0-ZK6Q6P5lvuiy7R0'
url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

document_info = []

while url:
    response = requests.get(url, headers=sp_headers).json()
    items = response.get('value', [])
    
    for record in items:
        internal_record = {
            'id': record.get('id'),
            'filename': record.get('name'),
            'createdDateTime': record.get('createdDateTime'),
            'lastModifiedDateTime': record.get('lastModifiedDateTime'),
            'webUrl': record.get('webUrl')
        }
        document_info.append(internal_record)
    
    # Check for next page link
    url = response.get('@odata.nextLink')

# Convert to DataFrame
document_df = pd.DataFrame(document_info)

print(f"✅ Total documents fetched: {len(document_df)}")
print(document_df.head())


✅ Total documents fetched: 4837
                                   id      filename       createdDateTime  \
0  01IHJB5CZPHMI4VM7ATNCYQCBBE4F6ZBHV  10159327.PDF  2025-10-17T10:21:10Z   
1  01IHJB5C3UHT7NUYAVVBHYBZ5T3EZKKZLV  10159328.PDF  2025-10-17T10:21:10Z   
2  01IHJB5C7VCADXS2BXVBBI3SXXGTEGD3WW  10159329.PDF  2025-10-17T10:21:10Z   
3  01IHJB5C44HMPL4LPHSJDLEAZSJEI2KJTZ  10159330.PDF  2025-10-17T10:21:10Z   
4  01IHJB5CYDUFHXXV62SREKSNWT2GVLB3AQ  10159331.PDF  2025-10-17T10:21:10Z   

   lastModifiedDateTime                                             webUrl  
0  2025-10-17T10:21:11Z  https://envu.sharepoint.com/sites/EnvuSpecific...  
1  2025-10-17T10:21:11Z  https://envu.sharepoint.com/sites/EnvuSpecific...  
2  2025-10-17T10:21:11Z  https://envu.sharepoint.com/sites/EnvuSpecific...  
3  2025-10-17T10:21:11Z  https://envu.sharepoint.com/sites/EnvuSpecific...  
4  2025-10-17T10:21:11Z  https://envu.sharepoint.com/sites/EnvuSpecific...  


In [72]:
document_df = document_df[['id','webUrl']]
document_df

,id,webUrl
0,01IHJB5CZPHMI4VM7ATNCYQCBBE4F6ZBHV,https://envu.sharepoint.com/sites/EnvuSpecific...
1,01IHJB5C3UHT7NUYAVVBHYBZ5T3EZKKZLV,https://envu.sharepoint.com/sites/EnvuSpecific...
2,01IHJB5C7VCADXS2BXVBBI3SXXGTEGD3WW,https://envu.sharepoint.com/sites/EnvuSpecific...
3,01IHJB5C44HMPL4LPHSJDLEAZSJEI2KJTZ,https://envu.sharepoint.com/sites/EnvuSpecific...
4,01IHJB5CYDUFHXXV62SREKSNWT2GVLB3AQ,https://envu.sharepoint.com/sites/EnvuSpecific...
...,...,...
4832,01IHJB5C2JGKBYILRTXRHKE7THLRI2NE4A,https://envu.sharepoint.com/sites/EnvuSpecific...
4833,01IHJB5C5DWFX4CYCFBJHKCBTJ2ENH7DYY,https://envu.sharepoint.com/sites/EnvuSpecific...
4834,01IHJB5C3U43BICW5JYZC2AMO6XQXHDTNY,https://envu.sharepoint.com/sites/EnvuSpecific...
4835,01IHJB5C6JUGNNKLBPMNCZ2MCVT64CTKTV,https://envu.sharepoint.com/sites/EnvuSpecific...


MetaData of Files

In [69]:
import requests
import pandas as pd
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Your existing document DataFrame with 'id' column
ids = document_df['id'].tolist()

drive_id = 'b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRFiYhc0-ZK6Q6P5lvuiy7R0'
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Function to fetch one file’s metadata
def fetch_fields(item_id):
    fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
    try:
        response = requests.get(fields_url, headers=sp_headers, timeout=10)
        if response.status_code == 200:
            fields = response.json()
            return {
                'id': item_id,
                'SpecificationNumber': fields.get('SpecificationNumber'),
                'filename': fields.get('FileName'),
                'DocumentType': fields.get('DocumentType'),
                'DocumentDescription': fields.get('DocumentDescription'),
                'DocumentStatus': fields.get('DocumentStatus')
            }
        else:
            return {'id': item_id, 'error': response.status_code}
    except Exception as e:
        return {'id': item_id, 'error': str(e)}

# Use a thread pool for concurrency
meta_data = []
with ThreadPoolExecutor(max_workers=20) as executor:  # You can tune max_workers (10–30 works well)
    futures = {executor.submit(fetch_fields, i): i for i in ids}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching metadata"):
        meta_data.append(future.result())

# Convert to DataFrame
meta_data_df = pd.DataFrame(meta_data)
print(meta_data_df.head())


Fetching metadata: 100%|██████████| 4837/4837 [02:00<00:00, 40.30it/s]

                                   id SpecificationNumber  filename  \
0  01IHJB5CZPHMI4VM7ATNCYQCBBE4F6ZBHV        102000009440  10159327   
1  01IHJB5C2U2VSFYWWUUNF2SKPJYVJPOGNT        102000009318  10159342   
2  01IHJB5C3UHT7NUYAVVBHYBZ5T3EZKKZLV        102000009442  10159328   
3  01IHJB5C6SB5PAL3OLDFBI6ZFZQRV34BES        102000009527  10159332   
4  01IHJB5C7VCADXS2BXVBBI3SXXGTEGD3WW        102000009440  10159329   

     DocumentType DocumentDescription DocumentStatus  
0  ZSpec_IBDV_URL            10159327       Approved  
1  ZSpec_IBDV_URL            10159342       Approved  
2  ZSpec_IBDV_URL            10159328       Approved  
3  ZSpec_IBDV_URL            10159332       Approved  
4  ZSpec_IBDV_URL            10159329       Approved  


In [81]:
merge_df = pd.merge(document_df,meta_data_df,on='id',how='left')


In [82]:
merge_df = merge_df[['SpecificationNumber','filename','DocumentType','DocumentDescription','DocumentStatus','webUrl']]
merge_df.rename(columns={'filename': 'FileName', 'webUrl': 'Document URL'}, inplace=True)
merge_df.to_excel('SharepointLinkUpdate.xlsx',index=False)

In [85]:
merge_df['SpecificationNumber'].duplicated()

0       False
1       False
2        True
3       False
4        True
        ...  
4832     True
4833     True
4834    False
4835     True
4836    False
Name: SpecificationNumber, Length: 4837, dtype: bool

In [47]:
DRIVE_ID = 'b!wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'
item_id = '01S4YKXPKXVUBUPL56UFEK3Z7ZCE3KBNUP'
update_body = {
    "specificationmainversion":'testing',
    "SpecificationNumber":"testing",
    "FileName":"testing",
    "DocumentType":"testing",
    "DocumentDescription":"testing",
    "DocId":"testing",
    "DocumentStatus":"testing"
}

update_url = f"https://graph.microsoft.com/v1.0/drives/{DRIVE_ID}/items/{item_id}/listItem/fields"
update_headers={
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'   # must be JSON
    }
update_resp = requests.patch(update_url, headers=update_headers, json=update_body, timeout=60)

In [48]:
update_resp.json()

{'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#drives('b%21wo3sBA26VUyRp2CMZopLbM3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q')/items('01S4YKXPKXVUBUPL56UFEK3Z7ZCE3KBNUP')/listItem/fields/$entity",
 '@odata.etag': '"4703ad57-beaf-48a1-ade7-f91136a0b68f,2"',
 'FileLeafRef': 'CERTIFICATE OF ANALYSIS OF TECHNICAL SUBSTANCE .docx',
 'DocId': 'testing',
 'FileName': 'testing',
 'SpecificationNumber': 'testing',
 'DocumentType': 'testing',
 'DocumentDescription': 'testing',
 'DocumentStatus': 'testing',
 'specificationmainversion': 'testing',
 'id': '681',
 'ContentType': 'Document',
 'Created': '2025-10-16T18:30:22Z',
 'AuthorLookupId': '15',
 'Modified': '2025-10-16T18:30:49Z',
 'EditorLookupId': '1073741822',
 '_CheckinComment': '',
 'LinkFilenameNoMenu': 'CERTIFICATE OF ANALYSIS OF TECHNICAL SUBSTANCE .docx',
 'LinkFilename': 'CERTIFICATE OF ANALYSIS OF TECHNICAL SUBSTANCE .docx',
 'DocIcon': 'docx',
 'FileSizeDisplay': '29891',
 'ItemChildCount': '0',
 'FolderChildCount': 

In [16]:
import requests
from tqdm import tqdm

def get_all_ids_with_next_page():
    query = (
        "SELECT id, name__v, filename__v, specification_main_version__c, specification_minor_version__c, subtype__v, status__v "
        "FROM documents "
        "WHERE (specification_minor_version__c != null OR specification_main_version__c != null) "
        "AND (status__v = 'Approved' OR status__v = 'Obsolete') "
        "AND (subtype__v = 'Formulation Inert Specification' "
        "OR subtype__v = 'Composition of Formulation (CoF)' "
        "OR subtype__v = 'Specification Technical Material' "
        "OR subtype__v = 'Formulation Specification' "
        "OR subtype__v = 'Selection List')"
    )

    base_url = 'https://envu-regulatoryone.veevavault.com/api/v24.2/query'
    params = {"q": query, "start": 0, "limit": 1000}
    print(params)
    
    all_ids = []
    
    # First request
    response = requests.post(url=base_url, headers=veeva_headers, params=params).json()
    
    total_records = response.get('responseDetails', {}).get('total', 0)
    print(f"Total records to fetch: {total_records}")

    pbar = tqdm(total=total_records, desc="Fetching document IDs")

    while True:
        data = response.get('data', [])
        all_ids.extend([r['id'] for r in data if 'id' in r])
        pbar.update(len(data))
        
        # Check if next_page exists
        next_page = response.get('responseDetails', {}).get('next_page')
        if not next_page:
            break

        # Fetch next page
        response = requests.get(url='https://envu-regulatoryone.veevavault.com' + next_page,
                                headers=veeva_headers).json()

    pbar.close()
    print(f"Total IDs collected: {len(all_ids)}")
    return all_ids

all_doc_ids = get_all_ids_with_next_page()


{'q': "SELECT id, name__v, filename__v, specification_main_version__c, specification_minor_version__c, subtype__v, status__v FROM documents WHERE (specification_minor_version__c != null OR specification_main_version__c != null) AND (status__v = 'Approved' OR status__v = 'Obsolete') AND (subtype__v = 'Formulation Inert Specification' OR subtype__v = 'Composition of Formulation (CoF)' OR subtype__v = 'Specification Technical Material' OR subtype__v = 'Formulation Specification' OR subtype__v = 'Selection List')", 'start': 0, 'limit': 1000}
Total records to fetch: 4845


Fetching document IDs: 100%|██████████| 4845/4845 [00:02<00:00, 2176.02it/s]

Total IDs collected: 4845


In [18]:
len(all_doc_ids)

4845

In [29]:
import requests

# access_token = "<YOUR_ACCESS_TOKEN>"
hostname = "envu.sharepoint.com"
site_name = "EnvuSpecificationsProd"

url = f"https://graph.microsoft.com/v1.0/sites/{hostname}:/sites/{site_name}"
headers = {"Authorization": f"Bearer {access_token}"}

response = requests.get(url, headers=sp_headers).json()
site_id = response.get("id")
print("Site ID:", site_id)


Site ID: envu.sharepoint.com,1a918a34-9289-4296-beb1-5cbfa7c3b776,a75fe4cd-dd0e-4a4e-bc67-9c913df28d11


In [34]:
import requests

# access_token = "<YOUR_ACCESS_TOKEN>"
site_id = "envu.sharepoint.com,1a918a34-9289-4296-beb1-5cbfa7c3b776,a75fe4cd-dd0e-4a4e-bc67-9c913df28d11"
drive_id = "b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q"



# Get all items
items_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root/children"
print(items_url)
response = requests.get(items_url, headers=sp_headers).json()
items = response.get("value", [])


https://graph.microsoft.com/v1.0/sites/envu.sharepoint.com,1a918a34-9289-4296-beb1-5cbfa7c3b776,a75fe4cd-dd0e-4a4e-bc67-9c913df28d11/drives/b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q/root/children


In [35]:
for item in items:
    item_id = item['id']
    del_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/items/{item_id}"
    r = requests.delete(del_url, headers=sp_headers)
    if r.status_code == 204:
        print(f"Deleted {item['name']}")


Deleted 10160254_(6).PDF
Deleted 10160254_(7).PDF
Deleted 10160254_(8).PDF
Deleted 10160254_(9).PDF
Deleted 10160255.PDF
Deleted 10160255_(1).PDF
Deleted 10160255_(10).PDF
Deleted 10160255_(11).PDF
Deleted 10160255_(2).PDF
Deleted 10160255_(3).PDF
Deleted 10160255_(4).PDF
Deleted 10160255_(5).PDF
Deleted 10160255_(6).PDF
Deleted 10160255_(7).PDF
Deleted 10161858_(10).PDF
Deleted 10161860_(6).PDF
Deleted 10161860_(7).PDF
Deleted 10161860_(8).PDF
Deleted 10161860_(9).PDF
Deleted 10161861.PDF
Deleted 10161861_(1).PDF
Deleted 10161861_(10).PDF
Deleted 10161861_(11).PDF
Deleted 10161861_(2).PDF
Deleted 10161861_(3).PDF
Deleted 10161861_(4).PDF
Deleted 10161861_(5).PDF
Deleted 10161861_(6).PDF
Deleted 10161861_(7).PDF
Deleted 10161861_(8).PDF
Deleted 10161861_(9).PDF
Deleted 10161862.PDF
Deleted 10161862_(1).PDF
Deleted 10161862_(10).PDF
Deleted 10161862_(11).PDF
Deleted 10161862_(2).PDF
Deleted 10161862_(3).PDF
Deleted 10161862_(4).PDF
Deleted 10161862_(5).PDF
Deleted 10161862_(6).PDF
Delet